In [2]:
import os
import random
import sys
#sys.path.append('../../../') 

import numpy as np
import torch as th
from torch import nn
from torch import optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.autograd import grad
import learn2learn as l2l
from learn2learn.data.transforms import NWays, KShots, LoadData, RemapLabels, ConsecutiveLabels

In [3]:
#def transformer_loss(L0,L1):
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)

def transformbatch(batch, transformer):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)
    data_transformed = transformer(data)
    return [data_transformed, labels]

def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = th.zeros(data.size(0)).byte()
    adaptation_indices[th.arange(shots*ways) * 2] = 1
    print(adaptation_indices.size())
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    print(adaptation_labels.size())
    evaluation_data, evaluation_labels = data[1 - adaptation_indices], labels[1 - adaptation_indices]
    
    #before adaptation, get the needed parameters for adaptation of transformer
    L0 = loss(learner(adaptation_data), adaptation_labels)
    L1 = loss(learner(evaluation_data), evaluation_labels)
     # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        train_error /= len(adaptation_data)
        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
   # print(valid_error)
    valid_error /= len(evaluation_data)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy, L0, L1

In [4]:
#variables to change
train_dataset = l2l.vision.datasets.MiniImagenet(root='../data', mode='train')
valid_dataset = l2l.vision.datasets.MiniImagenet(root='../data', mode='validation')
test_dataset = l2l.vision.datasets.MiniImagenet(root='../data', mode='test')
train_dataset = l2l.data.MetaDataset(train_dataset)
valid_dataset = l2l.data.MetaDataset(valid_dataset)
test_dataset = l2l.data.MetaDataset(test_dataset)

In [5]:
ways=5
shots=5
meta_lr=0.003
fast_lr=0.5
meta_batch_size=32
adaptation_steps=1
num_iterations=5
cuda=True
seed=42

In [6]:
random.seed(seed)
np.random.seed(seed)
th.manual_seed(seed)
device = th.device('cpu')
if cuda and th.cuda.device_count():
    th.cuda.manual_seed(seed)
    device = th.device('cuda')

In [7]:
train_transforms = [NWays(train_dataset, ways),KShots(train_dataset, 2*shots),LoadData(train_dataset),
        RemapLabels(train_dataset),
        ConsecutiveLabels(train_dataset),
    ]
train_tasks = l2l.data.TaskDataset(train_dataset, task_transforms=train_transforms,num_tasks=20000)

In [8]:
valid_transforms = [NWays(valid_dataset, ways), KShots(valid_dataset, 2*shots), LoadData(valid_dataset),
        ConsecutiveLabels(train_dataset),
        RemapLabels(valid_dataset),
    ]
valid_tasks = l2l.data.TaskDataset(valid_dataset, task_transforms=valid_transforms, num_tasks=600)

In [9]:
test_transforms = [NWays(test_dataset, ways), KShots(test_dataset, 2*shots), LoadData(test_dataset), 
        RemapLabels(test_dataset),
        ConsecutiveLabels(train_dataset),
    ]
test_tasks = l2l.data.TaskDataset(test_dataset, task_transforms=test_transforms, num_tasks=600)

In [10]:
# Create models, 2 networks, one is the transformer (conditioned on gradients of minibatches)
# the other is the meta-learner doing MAML
embedding = 64
#same as miniImagenetCNN to learn the latent space/conditioned on the gradients,
#used size 64 like LEO encoder
transformer = l2l.vision.models.MiniImagenetCNN(embedding)
transformer.to(device)
metal = l2l.vision.models.LinearBlock(embedding, ways)
metal.to(device)
maml = l2l.algorithms.MAML(metal, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
opt_transform = optim.Adam(transformer.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')
print(transformer)
print(metal)

MiniImagenetCNN(
  (base): ConvBase(
    (0): ConvBlock(
      (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (normalize): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ConvBlock(
      (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (normalize): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): ConvBlock(
      (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (normalize): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1),

In [ ]:
 for iteration in range(num_iterations):
        opt.zero_grad()
        meta_train_error = 0.0
        meta_train_accuracy = 0.0
        meta_valid_error = 0.0
        meta_valid_accuracy = 0.0
        meta_test_error = 0.0
        meta_test_accuracy = 0.0
        for task in range(meta_batch_size):
            # Compute meta-training loss
            learner = maml.clone()
            batch = train_tasks.sample()
            print(batch[0].size())
            batch_transformed = transformbatch(batch, transformer)
            evaluation_error, evaluation_accuracy, L0, L1 = fast_adapt(batch_transformed, learner, loss, adaptation_steps, shots, ways,
                           device)
            
            #check the gradients, torch.autogra.grad does not accumulate gradients ;)
          #  print("before")
           # for p in maml.parameters():
            #    print(p.grad)
            gradients0 = grad(L0, maml.parameters(), retain_graph=True, create_graph=True)
           # print("after")
            #for p in maml.parameters():
             #   print(p.grad)
            #print("gradients")
            #print(gradients0)
            gradients1 = grad(L1, maml.parameters(), retain_graph=True, create_graph=True)
            evaluation_error.backward()
            meta_train_error += evaluation_error.item()
            meta_train_accuracy += evaluation_accuracy.item()

            # Compute meta-validation loss
            learner = maml.clone()
            batch = valid_tasks.sample()
            batch_transformed = transformbatch(batch, transformer)
            evaluation_error, evaluation_accuracy, _, _ = fast_adapt(batch_transformed, learner,loss, adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            meta_valid_error += evaluation_error.item()
            meta_valid_accuracy += evaluation_accuracy.item()

            # Compute meta-testing loss
            learner = maml.clone()
            batch = test_tasks.sample()
            batch_transformed = transformbatch(batch, transformer)
            evaluation_error, evaluation_accuracy, _,_ = fast_adapt(batch_transformed, learner, loss, adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            meta_test_error += evaluation_error.item()
            meta_test_accuracy += evaluation_accuracy.item()

        # Print some metrics
        print('\n')
        print('Iteration', iteration)
        print('Meta Train Error', meta_train_error / meta_batch_size)
        print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
        print('Meta Valid Error', meta_valid_error / meta_batch_size)
        print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)
        print('Meta Test Error', meta_test_error / meta_batch_size)
        print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

        # Average the accumulated gradients and optimize., last step of meta-gradient
        for p in maml.parameters():
            p.grad.data.mul_(1.0 / meta_batch_size)
        opt.step()

torch.Size([50, 3, 84, 84])
torch.Size([50])
torch.Size([25])
before
None
None
None
None
after
None
None
None
None
gradients
(tensor([ 0.0022, -0.0689,  0.0215,  0.0557,  0.0846], device='cuda:0',
       grad_fn=<CudnnBatchNormBackwardBackward>), tensor([ 0.0045, -0.0644,  0.0225,  0.0511,  0.0427], device='cuda:0',
       grad_fn=<CudnnBatchNormBackwardBackward>), tensor([[-0.0594, -0.4706, -0.0256, -0.7164,  0.2068, -0.2853,  0.0563, -0.2484,
         -0.2776, -0.1635, -0.1548, -0.4708,  0.6861, -0.8108, -0.0441, -0.5403,
         -0.4345,  0.3895, -1.3378,  0.1365,  0.8986,  0.7473, -0.2026, -0.2166,
          0.3864,  0.0314, -0.3231,  0.4612,  1.2349,  0.2354, -0.0862, -0.1428,
         -0.2070, -1.0633, -1.1912,  0.0122, -0.3351, -0.4603,  0.1450,  0.0349,
          0.6610, -0.6435, -0.8402,  1.2083,  0.1899, -0.3261,  0.0086, -0.2803,
         -0.0327, -0.2220, -0.5531,  0.2499,  0.5999, -0.4719, -0.5790,  0.9992,
         -0.1266,  0.0381,  0.4507, -0.0801, -0.3023,  0.2652,  0

/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_15730493068

torch.Size([50])
torch.Size([25])
torch.Size([50])
torch.Size([25])


/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


torch.Size([50, 3, 84, 84])
torch.Size([50])
torch.Size([25])
before
tensor([-0.0013, -0.0038,  0.0027,  0.0002,  0.0043], device='cuda:0')
tensor([-0.0007, -0.0016,  0.0016, -0.0009, -0.0002], device='cuda:0')
tensor([[ 1.2863e-02, -8.5229e-03, -3.4589e-03, -1.8406e-02,  4.2207e-02,
         -1.0658e-02,  1.1379e-02,  2.6383e-02, -1.8469e-02,  1.0257e-02,
         -2.0640e-02, -2.5471e-02,  3.8182e-02, -3.4612e-02, -2.3995e-02,
         -7.5102e-03, -1.3458e-02,  1.9898e-02, -7.9386e-02, -3.3519e-02,
          1.9469e-02,  3.7168e-02, -9.8294e-03,  1.3595e-02,  1.2333e-02,
          1.3435e-02, -1.0970e-02,  1.9749e-02,  7.3184e-02, -6.3881e-04,
         -1.3651e-02,  5.6752e-03, -9.2215e-03, -5.8860e-02, -2.5233e-02,
         -8.1557e-03, -1.5794e-02, -4.4173e-02, -1.3665e-02, -2.1507e-02,
          5.1636e-02, -1.8799e-02, -1.8702e-02,  3.1885e-02, -8.5048e-04,
         -5.6002e-02, -3.7429e-02, -2.4789e-02, -3.7624e-03, -3.3553e-02,
         -5.1792e-02,  3.3851e-02,  5.8055e-02, -

...

/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_15730493068

torch.Size([50, 3, 84, 84])
torch.Size([50])
torch.Size([25])
before
tensor([ 0.0027,  0.0005, -0.0007,  0.0080, -0.0002], device='cuda:0')
tensor([ 0.0023,  0.0020, -0.0028,  0.0056,  0.0041], device='cuda:0')
tensor([[ 0.0014, -0.0202,  0.0103,  0.0124,  0.0104,  0.0033, -0.0161, -0.0138,
         -0.0054, -0.0069,  0.0266,  0.0012,  0.0153, -0.0037, -0.0029, -0.0306,
         -0.0025,  0.0085, -0.0162,  0.0377, -0.0005, -0.0133, -0.0204, -0.0299,
         -0.0036, -0.0101,  0.0058, -0.0170,  0.0110, -0.0008,  0.0093, -0.0310,
          0.0081,  0.0184, -0.0072,  0.0125,  0.0040, -0.0101, -0.0165,  0.0020,
         -0.0020,  0.0162, -0.0110, -0.0279, -0.0031,  0.0316, -0.0010,  0.0464,
          0.0142,  0.0160, -0.0055, -0.0228, -0.0330,  0.0066,  0.0292, -0.0379,
          0.0091,  0.0117, -0.0065,  0.0215,  0.0016,  0.0004, -0.0101,  0.0048],
        [-0.0066,  0.0447, -0.0256, -0.0019, -0.0613,  0.0214,  0.0097, -0.0514,
          0.0128, -0.0173,  0.0266,  0.0391, -0.0542, -0.00

/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_15730493068

torch.Size([50])
torch.Size([25])


/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
